In [1]:
import numpy as np

np.random.seed(777)
batch_size = 32
goal_num_classes = 125
src_num_classes = 125
epochs = 35
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from os import listdir
import PIL
from PIL import Image
from keras.utils import to_categorical
import copy
import time

Using TensorFlow backend.


In [2]:

###Read Goal Data Set

ImagesPath="D:\\SomeData\\Caltech101\\Images\\"
classes = [f for f in listdir(ImagesPath)]
train_volume=0.7
size = 128,128
goal_x_train=[]
goal_x_test=[]
goal_y_train=[]
goal_y_test=[]

for i in range(0,len(classes)):
    images=listdir(ImagesPath+classes[i])
    for j in range(0,(int)(len(images)*train_volume)):
        im=Image.open(ImagesPath+classes[i]+"\\"+images[j])
        im=im.resize(size)
        im=im.convert('RGB')
        data=img_to_array(im)
        data = data/255
        goal_x_train.append(data)
        goal_y_train.append(np.uint8(i))
    for j in range((int)(len(images)*train_volume),len(images)):
        im=Image.open(ImagesPath+classes[i]+"\\"+images[j])
        im=im.resize(size)
        im=im.convert('RGB')
        data=img_to_array(im)
        data = data/255
        goal_x_test.append(data)
        goal_y_test.append(np.uint8(i))
goal_x_test=np.array(goal_x_test)
goal_x_train=np.array(goal_x_train)
goal_y_train=to_categorical(goal_y_train, num_classes=src_num_classes)
goal_y_test=to_categorical(goal_y_test, num_classes=src_num_classes)
print ("GOAL DATA READED")


###Read SRC Data Set

ImagesPath="D:\\SomeData\\256_ObjectCategories\\256_ObjectCategories\\"
classes = [f for f in listdir(ImagesPath)]
train_volume=0.7
size = 128,128
src_x_train=[]
src_x_test=[]
src_y_train=[]
src_y_test=[]

for i in range(0,len(classes)):
    images=listdir(ImagesPath+classes[i])
    for j in range(0,(int)(len(images)*train_volume)):
        im=Image.open(ImagesPath+classes[i]+"\\"+images[j])
        im=im.resize(size)
        im=im.convert('RGB')
        data=img_to_array(im)
        data = data/255
        src_x_train.append(data)
        src_y_train.append(np.uint8(i))
    for j in range((int)(len(images)*train_volume),len(images)):
        im=Image.open(ImagesPath+classes[i]+"\\"+images[j])
        im=im.resize(size)
        im=im.convert('RGB')
        data=img_to_array(im)
        data = data/255
        src_x_test.append(data)
        src_y_test.append(np.uint8(i))
src_x_test=np.array(src_x_test)
src_x_train=np.array(src_x_train)
src_y_train=to_categorical(src_y_train,num_classes=src_num_classes)
src_y_test=to_categorical(src_y_test,num_classes=src_num_classes)
print ("SRC DATA READED")

GOAL DATA READED
SRC DATA READED


In [3]:
###Configure TMP Model
model_tmp = Sequential()
# this applies 32 convolution filters of size 3x3 each.
model_tmp.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=goal_x_train.shape[1:]))
model_tmp.add(Activation('relu'))
model_tmp.add(Conv2D(32, (3, 3)))
model_tmp.add(Activation('relu'))
model_tmp.add(MaxPooling2D(pool_size=(2, 2)))
model_tmp.add(Dropout(0.25))

model_tmp.add(Conv2D(64, (3, 3), padding='same'))
model_tmp.add(Activation('relu'))
model_tmp.add(Conv2D(64, (3, 3)))
model_tmp.add(Activation('relu'))
model_tmp.add(MaxPooling2D(pool_size=(2, 2)))
model_tmp.add(Dropout(0.25))

model_tmp.add(Conv2D(128, (5, 5), padding='same'))
model_tmp.add(Activation('tanh'))
model_tmp.add(Conv2D(128, (5, 5)))
model_tmp.add(Activation('tanh'))
model_tmp.add(MaxPooling2D(pool_size=(2, 2)))
model_tmp.add(Dropout(0.25))

model_tmp.add(Flatten())
model_tmp.add(Dense(512))
model_tmp.add(Activation('tanh'))
model_tmp.add(Dropout(0.5))
model_tmp.add(Dense(goal_num_classes))
model_tmp.add(Activation('softmax'))


###Configure SRC Model
model_src = Sequential()
# this applies 32 convolution filters of size 3x3 each.
model_src.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=src_x_train.shape[1:]))
model_src.add(Activation('tanh'))
model_src.add(Conv2D(32, (3, 3)))
model_src.add(Activation('relu'))
model_src.add(MaxPooling2D(pool_size=(2, 2)))
model_src.add(Dropout(0.25))

model_src.add(Conv2D(64, (3, 3), padding='same'))
model_src.add(Activation('relu'))
model_src.add(Conv2D(64, (3, 3)))
model_src.add(Activation('relu'))
model_src.add(MaxPooling2D(pool_size=(2, 2)))
model_src.add(Dropout(0.25))

model_src.add(Conv2D(128, (5, 5), padding='same'))
model_src.add(Activation('tanh'))
model_src.add(Conv2D(128, (5, 5)))
model_src.add(Activation('tanh'))
model_src.add(MaxPooling2D(pool_size=(2, 2)))
model_src.add(Dropout(0.25))

model_src.add(Flatten())
model_src.add(Dense(512))
model_src.add(Activation('tanh'))
model_src.add(Dropout(0.5))
model_src.add(Dense(src_num_classes))
model_src.add(Activation('softmax'))

# save init weightts
for layer in model_src.layers:
    g=layer.get_config()
    h=layer.get_weights()
    if (g['name'] == 'dense_1'):
        src_init_weight_d1 = copy.deepcopy(h)
    if (g['name'] == 'dense_2'):
        src_init_weight_d2 = copy.deepcopy(h)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [4]:
###1th Experiment
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# Let's train the model using RMSprop
model_src.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

print(model_src.summary())

t0=time.time()
model_src.fit(src_x_train, src_y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(goal_x_test, goal_y_test),
              shuffle=True)
t1=time.time()
print('Time =',(t1-t0))

from keras import backend as K
import tensorflow as tf
K.clear_session()
sess = tf.Session()
K.set_session(sess)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 128, 128, 32)      896       
_________________________________________________________________
activation_9 (Activation)    (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 126, 126, 32)      9248      
_________________________________________________________________
activation_10 (Activation)   (None, 126, 126, 32)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 63, 63, 32)        0      

7733/7733 [==============================] - 19s - loss: 5.2946 - acc: 0.0149 - val_loss: 5.0159 - val_acc: 0.0030
Epoch 33/35
7733/7733 [==============================] - 19s - loss: 5.2938 - acc: 0.0124 - val_loss: 4.9729 - val_acc: 0.0102
Epoch 34/35
7733/7733 [==============================] - 19s - loss: 5.2934 - acc: 0.0125 - val_loss: 5.0431 - val_acc: 0.0038
Epoch 35/35
7733/7733 [==============================] - 19s - loss: 5.3106 - acc: 0.0113 - val_loss: 4.8333 - val_acc: 0.0049
Time = 682.0637037754059


In [8]:
###Configure TMP Model
model_tmp = Sequential()
# this applies 32 convolution filters of size 3x3 each.
model_tmp.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=goal_x_train.shape[1:]))
model_tmp.add(Activation('relu'))
model_tmp.add(Conv2D(32, (3, 3)))
model_tmp.add(Activation('relu'))
model_tmp.add(MaxPooling2D(pool_size=(2, 2)))
model_tmp.add(Dropout(0.25))

model_tmp.add(Conv2D(64, (3, 3), padding='same'))
model_tmp.add(Activation('relu'))
model_tmp.add(Conv2D(64, (3, 3)))
model_tmp.add(Activation('relu'))
model_tmp.add(MaxPooling2D(pool_size=(2, 2)))
model_tmp.add(Dropout(0.25))

model_tmp.add(Conv2D(128, (5, 5), padding='same'))
model_tmp.add(Activation('tanh'))
model_tmp.add(Conv2D(128, (5, 5)))
model_tmp.add(Activation('tanh'))
model_tmp.add(MaxPooling2D(pool_size=(2, 2)))
model_tmp.add(Dropout(0.25))

model_tmp.add(Flatten())
model_tmp.add(Dense(512))
model_tmp.add(Activation('tanh'))
model_tmp.add(Dropout(0.5))
model_tmp.add(Dense(goal_num_classes))
model_tmp.add(Activation('softmax'))


###Configure SRC Model
model_src = Sequential()
# this applies 32 convolution filters of size 3x3 each.
model_src.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=src_x_train.shape[1:]))
model_src.add(Activation('tanh'))
model_src.add(Conv2D(32, (3, 3)))
model_src.add(Activation('relu'))
model_src.add(MaxPooling2D(pool_size=(2, 2)))
model_src.add(Dropout(0.25))

model_src.add(Conv2D(64, (3, 3), padding='same'))
model_src.add(Activation('relu'))
model_src.add(Conv2D(64, (3, 3)))
model_src.add(Activation('relu'))
model_src.add(MaxPooling2D(pool_size=(2, 2)))
model_src.add(Dropout(0.25))

model_src.add(Conv2D(128, (5, 5), padding='same'))
model_src.add(Activation('tanh'))
model_src.add(Conv2D(128, (5, 5)))
model_src.add(Activation('tanh'))
model_src.add(MaxPooling2D(pool_size=(2, 2)))
model_src.add(Dropout(0.25))

model_src.add(Flatten())
model_src.add(Dense(512))
model_src.add(Activation('tanh'))
model_src.add(Dropout(0.5))
model_src.add(Dense(src_num_classes))
model_src.add(Activation('softmax'))

In [9]:
###2th Experiment
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# Let's train the model using RMSprop
model_src.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

print(model_src.summary())

t0=time.time()
model_src.fit(goal_x_train, goal_y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(goal_x_test, goal_y_test),
              shuffle=True)
t1=time.time()
print('Time =',(t1-t0))

from keras import backend as K
import tensorflow as tf
K.clear_session()
sess = tf.Session()
K.set_session(sess)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 128, 128, 32)      896       
_________________________________________________________________
activation_9 (Activation)    (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 126, 126, 32)      9248      
_________________________________________________________________
activation_10 (Activation)   (None, 126, 126, 32)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 63, 63, 64)        18496     
__________

In [10]:
###Configure TMP Model
model_tmp = Sequential()
# this applies 32 convolution filters of size 3x3 each.
model_tmp.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=goal_x_train.shape[1:]))
model_tmp.add(Activation('relu'))
model_tmp.add(Conv2D(32, (3, 3)))
model_tmp.add(Activation('relu'))
model_tmp.add(MaxPooling2D(pool_size=(2, 2)))
model_tmp.add(Dropout(0.25))

model_tmp.add(Conv2D(64, (3, 3), padding='same'))
model_tmp.add(Activation('relu'))
model_tmp.add(Conv2D(64, (3, 3)))
model_tmp.add(Activation('relu'))
model_tmp.add(MaxPooling2D(pool_size=(2, 2)))
model_tmp.add(Dropout(0.25))

model_tmp.add(Conv2D(128, (5, 5), padding='same'))
model_tmp.add(Activation('tanh'))
model_tmp.add(Conv2D(128, (5, 5)))
model_tmp.add(Activation('tanh'))
model_tmp.add(MaxPooling2D(pool_size=(2, 2)))
model_tmp.add(Dropout(0.25))

model_tmp.add(Flatten())
model_tmp.add(Dense(512))
model_tmp.add(Activation('tanh'))
model_tmp.add(Dropout(0.5))
model_tmp.add(Dense(goal_num_classes))
model_tmp.add(Activation('softmax'))


###Configure SRC Model
model_src = Sequential()
# this applies 32 convolution filters of size 3x3 each.
model_src.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=src_x_train.shape[1:]))
model_src.add(Activation('tanh'))
model_src.add(Conv2D(32, (3, 3)))
model_src.add(Activation('relu'))
model_src.add(MaxPooling2D(pool_size=(2, 2)))
model_src.add(Dropout(0.25))

model_src.add(Conv2D(64, (3, 3), padding='same'))
model_src.add(Activation('relu'))
model_src.add(Conv2D(64, (3, 3)))
model_src.add(Activation('relu'))
model_src.add(MaxPooling2D(pool_size=(2, 2)))
model_src.add(Dropout(0.25))

model_src.add(Conv2D(128, (5, 5), padding='same'))
model_src.add(Activation('tanh'))
model_src.add(Conv2D(128, (5, 5)))
model_src.add(Activation('tanh'))
model_src.add(MaxPooling2D(pool_size=(2, 2)))
model_src.add(Dropout(0.25))

model_src.add(Flatten())
model_src.add(Dense(512))
model_src.add(Activation('tanh'))
model_src.add(Dropout(0.5))
model_src.add(Dense(src_num_classes))
model_src.add(Activation('softmax'))

In [11]:
###3th Experiment
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# Let's train the model using RMSprop
model_src.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

print(model_src.summary())

t0=time.time()
model_src.fit(src_x_train, src_y_train,
              batch_size=batch_size,
              epochs=epochs)
t1=time.time()
print('Time =',(t1-t0))

for layer in model_src.layers:
    g=layer.get_config()
    if ((g['name'] != 'dense_2')&(g['name'] != 'dense_2')):
        layer.trainable = False

t0=time.time()
model_src.fit(goal_x_train, goal_y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(goal_x_test, goal_y_test),
              shuffle=True)
t1=time.time()
print('Time =',(t1-t0))

from keras import backend as K
import tensorflow as tf
K.clear_session()
sess = tf.Session()
K.set_session(sess)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 128, 128, 32)      896       
_________________________________________________________________
activation_9 (Activation)    (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 126, 126, 32)      9248      
_________________________________________________________________
activation_10 (Activation)   (None, 126, 126, 32)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 63, 63, 64)        18496     
__________

6026/6026 [==============================] - 15s - loss: 4.8577 - acc: 0.0637 - val_loss: 4.5842 - val_acc: 0.0947
Epoch 12/35
6026/6026 [==============================] - 15s - loss: 4.8110 - acc: 0.0750 - val_loss: 4.2921 - val_acc: 0.1769
Epoch 13/35
6026/6026 [==============================] - 15s - loss: 4.8108 - acc: 0.0850 - val_loss: 4.2676 - val_acc: 0.1479
Epoch 14/35
6026/6026 [==============================] - 15s - loss: 4.7581 - acc: 0.0954 - val_loss: 4.1598 - val_acc: 0.1392
Epoch 15/35
6026/6026 [==============================] - 15s - loss: 4.7090 - acc: 0.1055 - val_loss: 4.2546 - val_acc: 0.1241
Epoch 16/35
6026/6026 [==============================] - 15s - loss: 4.5877 - acc: 0.1349 - val_loss: 4.0842 - val_acc: 0.1384
Epoch 17/35
6026/6026 [==============================] - 15s - loss: 4.3346 - acc: 0.1739 - val_loss: 4.3341 - val_acc: 0.1124
Epoch 18/35
6026/6026 [==============================] - 15s - loss: 5.0325 - acc: 0.0604 - val_loss: 4.4651 - val_acc: 0.1

In [12]:
###Configure TMP Model
model_tmp = Sequential()
# this applies 32 convolution filters of size 3x3 each.
model_tmp.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=goal_x_train.shape[1:]))
model_tmp.add(Activation('relu'))
model_tmp.add(Conv2D(32, (3, 3)))
model_tmp.add(Activation('relu'))
model_tmp.add(MaxPooling2D(pool_size=(2, 2)))
model_tmp.add(Dropout(0.25))

model_tmp.add(Conv2D(64, (3, 3), padding='same'))
model_tmp.add(Activation('relu'))
model_tmp.add(Conv2D(64, (3, 3)))
model_tmp.add(Activation('relu'))
model_tmp.add(MaxPooling2D(pool_size=(2, 2)))
model_tmp.add(Dropout(0.25))

model_tmp.add(Conv2D(128, (5, 5), padding='same'))
model_tmp.add(Activation('tanh'))
model_tmp.add(Conv2D(128, (5, 5)))
model_tmp.add(Activation('tanh'))
model_tmp.add(MaxPooling2D(pool_size=(2, 2)))
model_tmp.add(Dropout(0.25))

model_tmp.add(Flatten())
model_tmp.add(Dense(512))
model_tmp.add(Activation('tanh'))
model_tmp.add(Dropout(0.5))
model_tmp.add(Dense(goal_num_classes))
model_tmp.add(Activation('softmax'))


###Configure SRC Model
model_src = Sequential()
# this applies 32 convolution filters of size 3x3 each.
model_src.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=src_x_train.shape[1:]))
model_src.add(Activation('tanh'))
model_src.add(Conv2D(32, (3, 3)))
model_src.add(Activation('relu'))
model_src.add(MaxPooling2D(pool_size=(2, 2)))
model_src.add(Dropout(0.25))

model_src.add(Conv2D(64, (3, 3), padding='same'))
model_src.add(Activation('relu'))
model_src.add(Conv2D(64, (3, 3)))
model_src.add(Activation('relu'))
model_src.add(MaxPooling2D(pool_size=(2, 2)))
model_src.add(Dropout(0.25))

model_src.add(Conv2D(128, (5, 5), padding='same'))
model_src.add(Activation('tanh'))
model_src.add(Conv2D(128, (5, 5)))
model_src.add(Activation('tanh'))
model_src.add(MaxPooling2D(pool_size=(2, 2)))
model_src.add(Dropout(0.25))

model_src.add(Flatten())
model_src.add(Dense(512))
model_src.add(Activation('tanh'))
model_src.add(Dropout(0.5))
model_src.add(Dense(src_num_classes))
model_src.add(Activation('softmax'))

In [13]:
###4th Experiment
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# Let's train the model using RMSprop
for layer in model_src.layers:
    layer.trainable = True
model_src.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

print(model_src.summary())

t0=time.time()
model_src.fit(src_x_train, src_y_train,
              batch_size=batch_size,
              epochs=epochs)
t1=time.time()
print('Time =',(t1-t0))

weights = model_src.get_weights()
#move weights from src to tmp model

model_tmp.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
model_tmp.set_weights(weights)

t0=time.time()
model_tmp.fit(goal_x_train, goal_y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(goal_x_test, goal_y_test),
              shuffle=True)
t1=time.time()
print('Time =',(t1-t0))

from keras import backend as K
import tensorflow as tf
K.clear_session()
sess = tf.Session()
K.set_session(sess)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 128, 128, 32)      896       
_________________________________________________________________
activation_9 (Activation)    (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 126, 126, 32)      9248      
_________________________________________________________________
activation_10 (Activation)   (None, 126, 126, 32)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 63, 63, 64)        18496     
__________

6026/6026 [==============================] - 15s - loss: 4.9919 - acc: 0.0599 - val_loss: 4.4318 - val_acc: 0.0905
Epoch 12/35
6026/6026 [==============================] - 15s - loss: 4.9632 - acc: 0.0554 - val_loss: 4.4522 - val_acc: 0.0905
Epoch 13/35
6026/6026 [==============================] - 15s - loss: 4.9252 - acc: 0.0573 - val_loss: 4.7592 - val_acc: 0.0905
Epoch 14/35
6026/6026 [==============================] - 15s - loss: 4.9553 - acc: 0.0632 - val_loss: 4.5458 - val_acc: 0.0045
Epoch 15/35
6026/6026 [==============================] - 15s - loss: 4.9575 - acc: 0.0549 - val_loss: 4.5023 - val_acc: 0.0905
Epoch 16/35
6026/6026 [==============================] - 15s - loss: 4.9708 - acc: 0.0569 - val_loss: 4.4552 - val_acc: 0.0272
Epoch 17/35
6026/6026 [==============================] - 15s - loss: 4.9098 - acc: 0.0587 - val_loss: 4.7682 - val_acc: 0.0494
Epoch 18/35
6026/6026 [==============================] - 15s - loss: 4.9244 - acc: 0.0539 - val_loss: 4.5109 - val_acc: 0.0